# Chapter 7: Grouping for Aggregation, Filtration and Transformation
## Recipes
* [Defining an aggregation](#Defining-an-aggregation)
* [Grouping and aggregating with multiple columns and functions](#Grouping-and-aggregating-with-multiple-columns-and-functions)
* [Removing the MultiIndex after grouping](#Removing-the-MultiIndex-after-grouping)
* [Customizing an aggregation function](#Customizing-an-aggregation-function)
* [Customizing aggregating functions with \*args and \*\*kwargs](#Customizing-aggregating-functions-with-*args-and-**kwargs)
* [Examining the groupby object](#Examining-a-groupby-object)
* [Filtering for states with a minority majority](#Filtering-for-states-with-a-minority-majority)
* [Transforming through a weight loss bet](#Transforming-through-a-weight-loss)
* [Calculating weighted mean SAT scores per state with apply](#Calculating-weighted-mean-SAT-scores-per-state-with-apply)
* [Grouping by continuous variables](#Grouping-by-continuous-variables)
* [Counting the total number of flights between cities](#Counting-the-total-number-of-flights-between-cities)
* [Finding the longest streak of on-time flights](#Finding-the-longest-streak-of-on-time-flights)

In [2]:
import pandas as pd
import numpy as np

# Defining an aggregation

In [3]:
flights = pd.read_csv('data/flights.csv')

In [4]:
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [5]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean').head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [6]:
flights.groupby('AIRLINE').agg({'ARR_DELAY': 'mean'}).head()

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


In [7]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.mean).head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

# How it works ......

In [8]:
grouped = flights.groupby('AIRLINE')
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [9]:
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

## There's more ........

In [10]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.sqrt)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:844: RuntimeWarning: invalid value encountered in sqrt
  f = lambda x: func(x, *args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\groupby\generic.py:907: RuntimeWarning: invalid value encountered in sqrt
  output = func(group, *args, **kwargs)


Exception: Must produce aggregated value

# Grouping and aggregating with multiple columns and functions

In [2]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [3]:
# The number of cancelled flights for every airline per day weekday
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].agg('sum').head(7)

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
         6          21
         7          29
Name: CANCELLED, dtype: int64

In [5]:
# Find the number and percentage of cancelled and dirverted flights 
# the number and percentage of cancelled flights and the average and variance of the airtime.

group_cols = ['ORG_AIR', 'DEST_AIR']
agg_dict = {'CANCELLED': ['sum', 'mean', 'size'],
           'AIR_TIME': ['mean', 'var', 'std']}

flights.groupby(group_cols).agg(agg_dict).head()
# flights.groupby(['ORG_AIR', 'DEST_AIR']).agg({'CANCELLED': ['sum', 'mean', 'size'],
#           'AIR_TIME': ['mean', 'var']}).head()

CANCELLED              AIR_TIME                     
                       sum mean size        mean        var       std
ORG_AIR DEST_AIR                                                     
ATL     ABE              0  0.0   31   96.387097  45.778495  6.765981
        ABQ              0  0.0   16  170.500000  87.866667  9.373722
        ABY              0  0.0   19   28.578947   6.590643  2.567225
        ACY              0  0.0    6   91.333333  11.466667  3.386247
        AEX              0  0.0   40   78.725000  47.332692  6.879876

# Removing the MultiIndex after grouping

In [6]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [7]:
airline_info = flights.groupby(['AIRLINE', 'WEEKDAY']).agg({'DIST' : ['sum', 'mean'], 'ARR_DELAY': ['min', 'max']}).astype(int)

In [9]:
airline_info.head(10)

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
        6        1265340  1124       -50  858
        7        1461906  1100       -49  626
AS      1         121478  1029       -28  145
        2         114681  1081       -38   86
        3         120150  1044       -45  106

In [10]:
level0 = airline_info.columns.get_level_values(0)
level0

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')

In [12]:
level1 = airline_info.columns.get_level_values(1)
level1

Index(['sum', 'mean', 'min', 'max'], dtype='object')

In [13]:
airline_info.columns = level0 + '_' + level1

In [14]:
airline_info.columns

Index(['DIST_sum', 'DIST_mean', 'ARR_DELAY_min', 'ARR_DELAY_max'], dtype='object')

In [15]:
airline_info.head(7)

DIST_sum  DIST_mean  ARR_DELAY_min  ARR_DELAY_max
AIRLINE WEEKDAY                                                   
AA      1         1455386       1139            -60            551
        2         1358256       1107            -52            725
        3         1496665       1117            -45            473
        4         1452394       1089            -46            349
        5         1427749       1122            -41            732
        6         1265340       1124            -50            858
        7         1461906       1100            -49            626

In [17]:
airline_info.reset_index().head(7)

,AIRLINE,WEEKDAY,DIST_sum,DIST_mean,ARR_DELAY_min,ARR_DELAY_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
5,AA,6,1265340,1124,-50,858
6,AA,7,1461906,1100,-49,626


### There's more ........

In [18]:
flights.groupby(['AIRLINE'], as_index=False)['DIST'].agg('mean').round(0)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0
5,F9,970.0
6,HA,2615.0
7,MQ,404.0
8,NK,1047.0
9,OO,511.0


In [19]:
flights.groupby(['AIRLINE'], as_index=False, sort=False)['DIST'].agg('mean')

,AIRLINE,DIST
0,WN,809.985626
1,UA,1230.918891
2,MQ,404.229041
3,AA,1114.347865
4,F9,969.593014
5,EV,460.237453
6,OO,511.239375
7,NK,1047.428100
8,US,1181.226625
9,AS,1065.884115


# Customizing an aggregation function

In [6]:
college = pd.read_csv('data/college.csv')
college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [8]:
college.groupby('STABBR')['UGDS'].agg(['mean', 'std']).round(1).head()

,mean,std
STABBR,,
AK,2493.2,4051.7
AL,2789.9,4657.9
AR,1644.1,3142.8
AS,1276.0,NaN
AZ,4130.5,14893.6


In [13]:
def max_deviation(s):
    std_score = (s-s.mean())/s.std()
    return std_score.abs().max()

In [15]:
college.groupby('STABBR')['UGDS'].agg(max_deviation).round(1).head()

STABBR
AK    2.6
AL    5.8
AR    6.3
AS    NaN
AZ    9.9
Name: UGDS, dtype: float64

## There's more .....

In [16]:
college.groupby('STABBR')['UGDS', 'SATVRMID', 'SATMTMID'].agg([max_deviation, 'mean', 'std']).round(1).head()

UGDS                       SATVRMID               \
       max_deviation    mean      std max_deviation   mean   std   
STABBR                                                             
AK               2.6  2493.2   4051.7           NaN  555.0   NaN   
AL               5.8  2789.9   4657.9           1.6  508.5  54.5   
AR               6.3  1644.1   3142.8           2.2  491.9  48.3   
AS               NaN  1276.0      NaN           NaN    NaN   NaN   
AZ               9.9  4130.5  14893.6           1.9  538.3  28.0   

            SATMTMID               
       max_deviation   mean   std  
STABBR                             
AK               NaN  503.0   NaN  
AL               1.8  504.3  58.9  
AR               2.3  515.9  38.8  
AS               NaN    NaN   NaN  
AZ               1.4  536.7  39.7

In [17]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID'].agg([max_deviation, 'mean', 'std']).round(1).head()

UGDS                      SATVRMID               \
                max_deviation    mean     std max_deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                  2.1  3508.9  4539.5           NaN    NaN   NaN   
       1                  1.1   123.3   132.9           NaN  555.0   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  514.9  56.5   
       1                  2.4   979.7   870.8           1.5  498.0  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  481.1  37.9   

                     SATMTMID               
                max_deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0

In [19]:
max_deviation.__name__ = 'Max Deviation'

In [20]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID']\
                .agg([max_deviation, 'mean', 'std']).round(1).head()

UGDS                      SATVRMID               \
                Max Deviation    mean     std Max Deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                  2.1  3508.9  4539.5           NaN    NaN   NaN   
       1                  1.1   123.3   132.9           NaN  555.0   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  514.9  56.5   
       1                  2.4   979.7   870.8           1.5  498.0  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  481.1  37.9   

                     SATMTMID               
                Max Deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0

# Customizing aggregating functions with *args and **kwargs

In [23]:
college = pd.read_csv('data/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])

In [25]:
import inspect
inspect.signature(grouped.agg)

<Signature (arg, *args, **kwargs)>

## How to do it .....

In [26]:
def pct_between_1_3k(s):
    return s.between(1000, 3000).mean()

In [27]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between_1_3k).head(9)

STABBR  RELAFFIL
AK      0           0.142857
        1           0.000000
AL      0           0.236111
        1           0.333333
AR      0           0.279412
        1           0.111111
AS      0           1.000000
AZ      0           0.096774
        1           0.000000
Name: UGDS, dtype: float64

In [31]:
def pct_between(s, low, high):
    return s.between(low, high).mean()

In [32]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, 1000, 10000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

## How it works .......

In [35]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, high= 10000, low=1000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

In [36]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, 1000, high= 10000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

## There's more .......